In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import pandas as pd
from glob import glob

In [ ]:
import os

# Adjust the path based on actual folder location
fake_folder = '/kaggle/input/videos/SDFVD Small-scale Deepfake Forgery Video Dataset/videos_fake'
real_folder = '/kaggle/input/videos/SDFVD Small-scale Deepfake Forgery Video Dataset/videos_real'

# Check if paths exist
print("Fake folder exists:", os.path.exists(fake_folder))
print("Real folder exists:", os.path.exists(real_folder))

In [ ]:
from glob import glob

fake_videos = sorted(glob(os.path.join(fake_folder, '*.mp4')))
real_videos = sorted(glob(os.path.join(real_folder, '*.mp4')))

print("Number of fake videos:", len(fake_videos))
print("Number of real videos:", len(real_videos))

In [ ]:
all_videos =  real_videos+fake_videos
labels = [0]*len(real_videos) + [1]*len(fake_videos)  # 1 = Fake, 0 = Real

# Example:
for video, label in zip(all_videos, labels):
    print(f"{video} -> {'Fake' if label == 1 else 'Real'}")

In [ ]:
frame_count = []
for video_file in all_videos:
  cap = cv2.VideoCapture(video_file)
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames are " , frame_count)
print("Total no of video: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

In [ ]:
output_video_folder = "/kaggle/working/output_videos"
os.makedirs(output_video_folder, exist_ok=True)

label_file_path = os.path.join(output_video_folder, "labels.csv")
label_entries = []


In [ ]:
!pip install facenet-pytorch --no-deps --quiet



In [ ]:
import os
import cv2
import numpy as np
import torch
from PIL import Image
from facenet_pytorch import MTCNN
from tqdm import tqdm

# Initialize MTCNN face detector (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)

def process_video_to_output(video_path, output_folder, label, max_frames=150, size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to PIL image and RGB
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Detect faces
        boxes, _ = mtcnn.detect(img)

        if boxes is not None and len(boxes) > 0:
            # Pick largest face
            areas = [(x2 - x1) * (y2 - y1) for x1, y1, x2, y2 in boxes]
            largest_idx = np.argmax(areas)
            x1, y1, x2, y2 = boxes[largest_idx].astype(int)

            # Crop and resize
            face_crop = frame[y1:y2, x1:x2]
            if face_crop.size == 0:
                continue
            face_crop = cv2.resize(face_crop, size)
            frames.append(face_crop)

            if len(frames) >= max_frames:
                break

    cap.release()

    if len(frames) == 0:
        print(f"Skipping: {video_name} (no face detected)")
        return None

    # Pad or trim
    if len(frames) < max_frames:
        frames += [frames[-1]] * (max_frames - len(frames))
    else:
        frames = frames[:max_frames]

    # Save video
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, f"{video_name}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 25.0, size)

    for frame in frames:
        out.write(frame)
    out.release()

    print(f"Saved: {output_path}")
    return video_name, label


In [ ]:
from tqdm import tqdm
import cv2
import os
from glob import glob
# No need to import CascadeClassifier if not used.
# from cv2 import CascadeClassifier # This import is not used and can be removed

fake_folder = '/kaggle/input/videos/SDFVD Small-scale Deepfake Forgery Video Dataset/videos_fake'
real_folder = '/kaggle/input/videos/SDFVD Small-scale Deepfake Forgery Video Dataset/videos_real'

fake_videos = sorted(glob(os.path.join(fake_folder, '*.mp4')))
real_videos = sorted(glob(os.path.join(real_folder, '*.mp4')))

all_video_paths = [(path, 1) for path in fake_videos] + [(path, 0) for path in real_videos]


for path, label in tqdm(all_video_paths):
    try:
        result = process_video_to_output(path, output_video_folder, label)
        if result:
            video_name, label = result
            label_entries.append((video_name, label))
    except Exception as e:
        # Print the path of the video file that caused the error
        print(f"\nError processing video: {path}")
        print(f"Error details: {e}")
        # Continue to the next video file
        continue

In [ ]:
df_labels = pd.DataFrame(label_entries, columns=['video_name', 'label'])
df_labels.to_csv(label_file_path, index=False)

In [ ]:
df_labels = df_labels.drop_duplicates()
df_labels = df_labels.reset_index(drop=True)


In [ ]:
X=df_labels['video_name']
y=df_labels['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
X_train

In [ ]:
from torchvision import transforms
im_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])


In [ ]:
video_label_dict = dict(zip(df_labels['video_name'], df_labels['label']))

In [ ]:
video_label_dict

In [ ]:
import torch
from torch.utils.data import Dataset
import cv2
import os
import numpy as np
from torchvision import transforms

class VideoDataSet(Dataset):
    def __init__(self, video_paths, label_dict, sequence_length=150, transform=None):
        self.video_paths = video_paths
        self.label_dict = label_dict
        self.seq_length = sequence_length
        self.transform = transform

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, index):
        path = self.video_paths[index]
        video_name = os.path.splitext(os.path.basename(path))[0]
        label = self.label_dict.get(video_name, -1)

        frames = self.extract_frames(path)

        if len(frames) == 0:
            dummy = torch.zeros((3, 224, 224))
            frames = [dummy] * self.seq_length

        video_tensor = torch.stack(frames)  # Shape: [seq_len, 3, 224, 224]
        return video_tensor, label

    def extract_frames(self, path):
        cap = cv2.VideoCapture(path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Uniform sampling indices
        indices = np.linspace(0, max(1, total_frames - 1), self.seq_length).astype(int)

        frames = []
        for i in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            success, frame = cap.read()
            if not success:
                break

            # Convert BGR to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Apply transform
            if self.transform:
                frame = self.transform(frame)
            else:
                # Default transformation if none provided
                frame = torch.from_numpy(cv2.resize(frame, (224, 224))).permute(2, 0, 1).float() / 255.0

            frames.append(frame)

        cap.release()

        # Pad if fewer frames
        while len(frames) < self.seq_length:
            frames.append(torch.zeros((3, 224, 224)))

        return frames


In [ ]:
X_train_dataset=VideoDataSet(X_train,video_label_dict,sequence_length=150,transform=train_transform)
X_test_dataset=VideoDataSet(X_test,video_label_dict,sequence_length=150,transform=test_transform)

In [ ]:
from torch.utils.data import DataLoader
y_train_dataset=DataLoader(X_train_dataset,batch_size=4,shuffle=True)
y_test_dataset=DataLoader(X_test_dataset,batch_size=4,shuffle=True)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(X_train_dataset, batch_size=4, shuffle=True)

for videos, labels in train_loader:
    print("Batch video tensor shape:", videos.shape)
    print("Batch labels:", labels)


In [ ]:
test_loader=DataLoader(X_test_dataset,batch_size=4,shuffle=True)
for videos, labels in test_loader:
    print("Batch video tensor shape:", videos.shape)
    print("Batch labels:", labels)

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn

# Load pretrained ResNeXt
resnext = models.resnext50_32x4d(pretrained=True)

# Remove final classification layer (we want 2048-dim features)
resnext.fc = nn.Identity()

# Set to eval mode
resnext.eval()
resnext.cuda()  # Or move to your device


In [ ]:
import torch
from torch import nn

class LSTMModel(nn.Module):
    def __init__(self, num_classes=1, latent_dim=2048, lstm_layers=1, hidden_dim=128, bidirectional=False, dropout=0.4):
        super(LSTMModel, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=bidirectional
        )
        
        self.dropout = nn.Dropout(dropout)
        
        lstm_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        
        self.linear1 = nn.Linear(lstm_output_dim, num_classes)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.LeakyReLU()
        
    def forward(self, x):
        # x shape: [batch_size, seq_length, latent_dim]
        
        # LSTM expects input [batch, seq, features]
        lstm_out, _ = self.lstm(x)  # [B, T, hidden_dim * directions]
        
        # Mean pool across the sequence dimension
        pooled = torch.mean(lstm_out, dim=1)  # [B, hidden_dim * directions]
        
        x = self.dropout(pooled)
        x = self.linear1(x)    # [B, num_classes]
        
        if self.linear1.out_features == 1:
            # Binary classification: apply sigmoid
            x = self.sigmoid(x).squeeze(1)  # [B]
        else:
            # Multi-class classification: maybe softmax outside
            pass
        
        return x


In [ ]:
@torch.no_grad()
def extract_resnext_features_batch(batch_videos, model, device='cuda'):
    """
    Args:
        batch_videos: Tensor [B, T, 3, 224, 224] (on CPU or GPU)
        model: ResNeXt model with fc=Identity(), already on device
        device: 'cuda' or 'cpu'

    Returns:
        features: Tensor [B, T, 2048] on the SAME device (GPU preferred)
    """
    model.eval()
    batch_videos = batch_videos.to(device)  # Move input to device (GPU)
    B, T, C, H, W = batch_videos.shape

    flat_videos = batch_videos.view(B * T, C, H, W)  # Flatten to [B*T, 3, 224, 224]
    features = model(flat_videos)                     # Extract features [B*T, 2048]
    features = features.view(B, T, -1)                # Reshape to [B, T, 2048]

    return features


In [ ]:
def train_model(model, train_loader, num_epochs, optimizer, device, resnext):
    model.to(device)
    model.train()

    criterion = torch.nn.BCELoss()  # For binary classification

    for epoch in range(num_epochs):
        total_loss = 0.0
        total_correct = 0
        total_samples = 0

        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        
        for batch_idx, (videos, labels) in enumerate(train_loader):
            labels = labels.to(device).float()

            optimizer.zero_grad()

            # Extract ResNeXt features: shape = [B, T, 2048]
            with torch.no_grad():  # Prevent gradient calc for ResNeXt
                features = extract_resnext_features_batch(videos, resnext, device=device)

            # Forward pass through LSTM classifier
            outputs = model(features).squeeze()  # Shape: [B]

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Binary predictions using threshold 0.5
            preds = (outputs >= 0.5).float()

            batch_correct = (preds == labels).sum().item()
            total_correct += batch_correct
            total_samples += labels.size(0)
            total_loss += loss.item()

            batch_accuracy = batch_correct / labels.size(0)
            running_accuracy = total_correct / total_samples

            print(f"Batch {batch_idx+1}: Loss={loss.item():.4f}, Batch Acc={batch_accuracy*100:.2f}%, Running Acc={running_accuracy*100:.2f}%")

        avg_loss = total_loss / (batch_idx + 1)
        final_accuracy = total_correct / total_samples
        print(f"Epoch {epoch+1} Summary: Avg Loss={avg_loss:.4f}, Accuracy={final_accuracy*100:.2f}%")


In [ ]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LSTMModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# assuming train_loader is ready with (videos, labels)

train_model(model, train_loader, num_epochs=10, optimizer=optimizer, device=device,resnext=resnext)


In [ ]:
def test(model, test_loader, resnext, device):
    model.eval()
    criterion = torch.nn.BCELoss()

    running_loss = 0.0
    running_correct = 0
    total_samples = 0

    print("Testing...\n")

    with torch.no_grad():
        batch_num = 0
        for videos, labels in test_loader:
            batch_num += 1

            # Move to device
            videos = videos.to(device)             # [B, T, C, H, W]
            labels = labels.float().to(device)     # [B]

            # Extract features using ResNeXt
            features = extract_resnext_features_batch(videos, resnext, device=device)  # [B, T, 2048]

            # Forward pass through LSTM-based model
            outputs = model(features).squeeze()    # [B], probabilities
            loss = criterion(outputs, labels)

            # Predictions
            preds = (outputs >= 0.5).float()       # [B]

            # Print predictions for each sample
            print(f"Batch {batch_num}:")
            for i in range(labels.size(0)):
                actual = int(labels[i].item())
                predicted = int(preds[i].item())
                probability = outputs[i].item()
                print(f"  Sample {total_samples + i + 1}: Predicted = {predicted}, Probability = {probability:.4f}, True = {actual}")

            # Accuracy metrics
            batch_correct = (preds == labels).sum().item()
            batch_accuracy = batch_correct / labels.size(0)
            running_correct += batch_correct
            total_samples += labels.size(0)
            running_accuracy = running_correct / total_samples
            running_loss += loss.item()

            print(f"  --> Loss={loss.item():.4f}, Batch Acc={batch_accuracy*100:.2f}%, Running Acc={running_accuracy*100:.2f}%\n")

    avg_loss = running_loss / batch_num
    print(f"\nTest Summary: Avg Loss={avg_loss:.4f}, Final Accuracy={running_accuracy*100:.2f}%")


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Call the test function
test(model, test_loader,resnext, device)